[Тут](https://colab.research.google.com/drive/1EpJkZLzM0v4c7A3O9XWtXds1_7xAreZx) весь оставшийся мусор по  подготовке датасетов

Пока: 


*   Каждые 10к датасета дают прибавление в accuracy (20к - 0.81; 70к - 0.87)
*   То, что выше - это получается на первых 2-3 эпохах, дальше модель переобучается. 
*   Если убрать стопслова из датасета на 50к записей, то результаты хуже (0.8 acc)
* Добавила регуляризацию: acc = 0.882 на пяти эпохах

Что делаю сейчас: 


*   Пинаю писюны





In [0]:
#!pip install transformers
!pip install tokenizers

In [0]:
!pip install transformers

In [0]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import spacy
from spacy.symbols import ORTH
import re
import json
from tqdm import tqdm_notebook
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec, KeyedVectors

import torch
from torchtext.data import Field, LabelField, BucketIterator, TabularDataset, Iterator, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.vocab import Vectors

#from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel
#from tokenizers import SentencePieceBPETokenizer


LENGTH = 10000
SEED = 42
np.random.seed(SEED)

import nltk
from nltk import tokenize
nltk.download('punkt')

#spacy_en = spacy.load('en')
#spacy_en.tokenizer.add_special_case("don't", [{ORTH: "do"}, {ORTH: "not"}])
#spacy_en.tokenizer.add_special_case("didn't", [{ORTH: "did"}, {ORTH: "not"}]) #adding special case so that tokenizer("""don't""") != 'do'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
tokenizer = SentencePieceBPETokenizer()
tokenizer.train(['/content/wiki_train.txt', '/content/wiki_valid.txt'], special_tokens=['<eos>', '<unk>', '<start>'], vocab_size=30000)
tokenizer

NameError: ignored

In [0]:
def clean_data(text):
    text = re.sub(r'\<[^>]*\>', '', text)
    text = re.sub(r'\<[^>]*\>', '', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'&[a-z]{0,7};', ' ', text)
    text = re.sub(r'\s{2,10}', ' ', text)
    text = re.sub(r'\s{2,10}', ' ', text)
    text = re.sub('\\x\d{1,4}', '', text)
    return text

In [0]:
def sent_first(text): # to gpt2
    return tokenize.sent_tokenize(text)[0]

def sent_splitter(text): # to classification model
    return ' '.join(tokenize.sent_tokenize(text)[:2])

def tokenizer(text):
    pass
    #return [tok.lemma_ for tok in spacy_en.tokenizer(text) if tok.text.isalpha()]
    

In [0]:
def open_file(file):
    with open(file, 'r', encoding='utf-8') as f:
        fake = [line for line in f.readlines()]
    return fake

# Process Original Dataset

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

idd = '1h_kUrWbKvZR0iRms8zKMUJO5zq9wbIXW' #Newsroom
downloaded_ = drive.CreateFile({'id':idd}) 
downloaded_.GetContentFile('train.jsonl')

In [0]:
def initial(file, start_idx=0):
    
    with open(file, 'r') as json_file:
        json_list = list(json_file)[start_idx:start_idx+LENGTH]
    
    data = []
    for json_str in json_list:
        result = json.loads(json_str)
        data.append(result['text'])

    df = pd.DataFrame()
    df['text'] = data
    df['label'] = ['real'] * len(data)
    df['text'] = df['text'].apply(clean_data)
    
    return df

In [0]:
def process_df(df):

    text_gpt2 = list(df['text'].apply(sent_first))[:LENGTH]
    df.text  = df['text'].apply(sent_splitter)

    return text_gpt2, df 

In [0]:
df = initial('train.jsonl', start_idx=0) #Выгружаю пачками по 10к данные, генерирую текст, добавляю в финальный датасет
text_gpt2, df = process_df(df)

In [0]:
with open('texts_to_gpt2.txt', 'w', encoding='utf-8') as f:
        for item in text_gpt2:
            f.write('{}\n'.format(item))

In [0]:
df.head()

In [0]:
len(text_gpt2), text_gpt2[0]

(10000,
 'HAMBURG, Germany, June 3 \x97 As he left the soccer field after a club match in the eastern German city of Halle on March 25, the Nigerian forward Adebowale Ogungbure was spit upon, jeered with racial remarks and mocked with monkey noises.')

In [0]:
text_gpt2[-1], df.text.tolist()[-1]

In [0]:
clean_data(text_gpt2[0])

error: ignored

# Data Generation

In [0]:
MAX_LENGTH = 25
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [0]:
def generate_sentence(text, tokenizer, model):
    generated = tokenizer.encode(text)
    initial_length = len(generated)
    context = torch.tensor([generated])
    past = None

    for i in range(MAX_LENGTH):
        output, past = model(context, past=past)
        token = torch.argmax(output[0, :])

        generated += [token.tolist()]
        context = token.unsqueeze(0)
        if (tokenizer.decode(token.tolist()) in ['!', '?', '.']) and (len(generated) - initial_length > 2):
            break
    sequence = tokenizer.decode(generated)
    return sequence

In [0]:
def make_fake(texts, fake=[], real=[], df=df, tokenizer=tokenizer, model=model):
    i = 0
    for ind, el in tqdm(enumerate(texts)):
        try:
            sent = generate_sentence(el, tokenizer=tokenizer, model=model)
            sent = re.sub(r'\n', '', sent)
            fake.append(sent)
            real.append(df.text.tolist()[ind])
        except:
            i+=1
        if i % 1000 == 0:
            print(i, 'out of', ind,' are not found in emb dict')    

    with open('fake.txt', 'w', encoding='utf-8') as f:
        for item in fake:
            f.write('{}\n'.format(item))
    with open('real.txt', 'w', encoding='utf-8') as f:
        for item in real:
            f.write('{}\n'.format(item))
    return fake, real

In [0]:
fake, real = make_fake(text_gpt2, fake=fake, real=real) #Если нет real и fake - убрать из параметров

In [0]:
len(fake), len(real)

In [0]:
#Запустить, если в датафрейме появляются лишние строчки с \n (всегда)
fake = [re.sub(r'\n', '', line) for line in fake]
real = [re.sub(r'\n', '', line) for line in real]
with open('fake.txt', 'w', encoding='utf-8') as f:
        for item in fake:
            f.write('{}\n'.format(item))
with open('real.txt', 'w', encoding='utf-8') as f:
    for item in real:
        f.write('{}\n'.format(item))

In [0]:
fake[0], real[0]

In [0]:
fake[-1], real[-1]

# To DF and CSV

In [0]:
def make_dataframe(df, fake, real, df_concat=None):
    
    fake = open_file(fake) if type(fake) == str else fake
    
    real = open_file(real) if type(real) == str else real
    new_df = pd.DataFrame()
    new_df['text'] = real + fake
    new_df['label'] = ['real'] * len(real) + ['fake'] * len(fake)
    
    if df_concat is not None:
       new_df = pd.concat([df_concat, new_df])
    return new_df

In [0]:
final_df = make_dataframe(df, 'fake.txt', 'real.txt')
final_df.to_csv("dataset.csv", index=False)

In [0]:
final_df.head()

In [0]:
final_df.tail()

# Data - Сразу сюда, когда есть final_df и dataset.csv

In [0]:
#Запускать если нужен готовый датафрейм (final_df) и файл dataset.csv, без генерации чего либо, сразу модель запускать
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

idd = '1goRkFXGEyrBoDuxfnDvmpHL3QQNM4AWK'
downloaded_ = drive.CreateFile({'id':idd}) 
downloaded_.GetContentFile('dataset.csv')

downloaded_ = drive.CreateFile({'id':'1HmV7mhKV7bS3TsEEgAe60oMoh8pvS7IS'}) 
downloaded_.GetContentFile('fake.txt')

downloaded_ = drive.CreateFile({'id':'1vRBk1xOkRumOavk1RZdTIFI_SebrLCBW'}) 
downloaded_.GetContentFile('real.txt')

fake = open_file('fake.txt')
real = open_file('real.txt')

final_df = pd.read_csv('dataset.csv')


In [0]:
def tokenizer(text):
    return [tok.lemma_ for tok in spacy_en.tokenizer(text) if (tok.text.isalpha() or tok.text.isdigit())]

In [0]:
text = list(final_df['text'].apply(tokenizer))
model = Word2Vec(text, size=100, iter=10) #building emb of size 100
model_weights = torch.FloatTensor(model.wv.vectors)
model.wv.save_word2vec_format('pretrained_embeddings')
vectors = Vectors(name='pretrained_embeddings', cache='./') #and saving the weights to build vocab later

In [0]:
MAX_VOCAB_SIZE = 50000
classes={'fake': 0, 'real': 1}


TEXT = Field(sequential=True, include_lengths=False, batch_first=True, tokenize=tokenizer, 
             pad_first=True, lower=True, eos_token='<eos>') 
LABEL = LabelField(dtype=torch.float, use_vocab=True, preprocessing=lambda x: classes[x])


dataset = TabularDataset('dataset.csv', 
                                format='csv', fields=[('text', TEXT), ('label',LABEL),], 
                                skip_header=True)

TEXT.build_vocab(dataset, vectors=vectors,
                   unk_init = torch.Tensor.normal_, max_size=MAX_VOCAB_SIZE, min_freq=2)
LABEL.build_vocab(dataset)
vocab = TEXT.vocab
print('Vocab size:', len(TEXT.vocab.itos))
TEXT.vocab.itos[:10]

In [0]:
train, test = dataset.split(0.8, stratified=True)
train, valid = train.split(0.8, stratified=True)

# Model  - simple BiLSTM

In [0]:
class MyModel(nn.Module):
    
    def __init__(self, vocab_size, embed_size, hidden_size):
        super(MyModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.embedding.weight.data.copy_(vocab.vectors)
        self.rnn = nn.LSTM(input_size=embed_size,
                           hidden_size=hidden_size,
                           bidirectional=True,
                           batch_first=True,
                          )
        
        self.fc = nn.Linear(hidden_size * 2 *2, 1)
        
    def forward(self, x):
        
        x = self.embedding(x)
           
        _, (hidden, cell) = self.rnn(x)
        
        hidden = hidden.transpose(0,1)
        cell = cell.transpose(0,1)
        hidden = hidden.contiguous().view(hidden.size(0),-1)
        cell = cell.contiguous().view(cell.size(0),-1)
        x = torch.cat([hidden, cell], dim=1).squeeze(1)
        x = self.fc(x)
        return x

In [0]:
batch_size = 256

model = MyModel(len(TEXT.vocab.itos),
                embed_size=100,
                hidden_size=128,
               )
model.embedding.weight.data.copy_(TEXT.vocab.vectors);

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train, valid, test),
    batch_sizes=(batch_size, batch_size, batch_size),
    shuffle=True,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
)

optimizer = optim.Adam(model.parameters(), weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, verbose=True, cooldown=5)
criterion = nn.BCEWithLogitsLoss()

In [0]:
def accuracy_score(preds, y):
    preds = torch.round(torch.sigmoid(preds))
    preds = (preds == y).float()
    accuracy = preds.sum() / len(preds)
    return accuracy

In [0]:
def test_model(model, test_iterator):
    test_acc = []

    with torch.no_grad():
        for item in test_iterator:
            x = item.text
            y = item.label
            preds = model(x).squeeze(1)
            test_acc.append(accuracy_score(preds, y))
    test_acc = np.mean(test_acc) 
    return np.mean(test_acc)

In [0]:
def train_cnn(model, train_iterator, valid_iterator, criterion, n_epochs=20):
    
    history = []

    for epoch in range(n_epochs):
        train_loss = []
        train_acc = []
        model.train()

    
        for item in tqdm(train_iterator):
            x = item.text
            y = item.label
            optimizer.zero_grad()
            preds = model(x).squeeze(1)
            loss = criterion(preds, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.data.detach().item())
            train_acc.append(accuracy_score(preds, y))

        train_loss = np.mean(train_loss)
        train_acc = np.mean(train_acc)

        model.eval()
        val_loss = []
        val_acc = []
        for item in valid_iterator:
            x = item.text
            y = item.label
            preds = model(x).squeeze(1)
            loss = criterion(preds, y)
            val_loss.append(loss.data.detach().item())
            val_acc.append(accuracy_score(preds, y))
        val_loss = np.mean(val_loss)
        val_acc = np.mean(val_acc)

        scheduler.step(val_loss)


        print('Epoch: {}. Train loss: {:.3f}. Train accuracy: {:.3f}. Val loss: {:.3f}. Val accuracy: {:.3f}'.format(
            epoch, train_loss, train_acc, val_loss, val_acc))        
        
        history.append({
            'epoch': epoch,
            'train_loss': train_loss,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'val_loss': val_loss
        })

        #if epoch % 5 == 0:
            #torch.save(model.state_dict(), '/content/model_test')

    return history

In [0]:
def clean_tqdm():
    for instance in list(tqdm._instances): 
        tqdm._decr_instances(instance)

In [0]:
clean_tqdm()
history = train_cnn(model, train_iterator, valid_iterator,
          criterion, n_epochs=5)

In [0]:
test_accuracy = test_model(model, test_iterator)
print('Test accuracy: {}'.format(np.mean(test_accuracy)))

In [0]:
plt.plot([x['val_loss'] for x in history], label='Val loss')
plt.plot([x['train_loss'] for x in history], label='Train loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [0]:
plt.plot([x['val_acc'] for x in history], label='Val acc')
plt.plot([x['train_acc'] for x in history], label='Train acc')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()